In [12]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [14]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

# Retriver and Chain with Langchain

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("bona.pdf")
text_documents = loader.load()
text_documents

[Document(page_content='DePaul Univ ersity DePaul Univ ersity \nDigital Commons@DeP aul Digital Commons@DeP aul \nNapoleon T ranslations French Pr ogram \n1-1-2009 \nProclomations, Speeches and Letters of Napoleon Buonapar te Proclomations, Speeches and Letters of Napoleon Buonapar te \nDuring His Campaign of E gypt 1-8 During His Campaign of E gypt 1-8 \nTrent Daile y-Chwalibog \nBrittany Gignac \nFollow this and additional works at: https:/ /via.libr ary.depaul.edu/napoleon \nRecommended Citation Recommended Citation \nDaile y-Chwalibog, T rent and Gignac, Brittany . (2009) Pr oclomations, Speeches and Letters of Napoleon \nBuonapar te During His Campaign of E gypt 1-8. \nhttps:/ /via.libr ary.depaul.edu/napoleon/6 \nThis T ranslation is br ought t o you for fr ee and open access b y the F rench Pr ogram at Digital Commons@DeP aul. It \nhas been accepted for inclusion in Napoleon T ranslations b y an authoriz ed administr ator of Digital \nCommons@DeP aul. F or mor e information, ple

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)
documents[:5]

[Document(page_content='DePaul Univ ersity DePaul Univ ersity \nDigital Commons@DeP aul Digital Commons@DeP aul \nNapoleon T ranslations French Pr ogram \n1-1-2009 \nProclomations, Speeches and Letters of Napoleon Buonapar te Proclomations, Speeches and Letters of Napoleon Buonapar te \nDuring His Campaign of E gypt 1-8 During His Campaign of E gypt 1-8 \nTrent Daile y-Chwalibog \nBrittany Gignac \nFollow this and additional works at: https:/ /via.libr ary.depaul.edu/napoleon \nRecommended Citation Recommended Citation \nDaile y-Chwalibog, T rent and Gignac, Brittany . (2009) Pr oclomations, Speeches and Letters of Napoleon \nBuonapar te During His Campaign of E gypt 1-8. \nhttps:/ /via.libr ary.depaul.edu/napoleon/6 \nThis T ranslation is br ought t o you for fr ee and open access b y the F rench Pr ogram at Digital Commons@DeP aul. It \nhas been accepted for inclusion in Napoleon T ranslations b y an authoriz ed administr ator of Digital', metadata={'source': 'bona.pdf', 'page': 0}),

In [4]:
from langchain_community.embeddings import OpenAIEmbeddings,OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [5]:
embeddings = OllamaEmbeddings(model="llama3")

In [7]:
db=FAISS.from_documents(documents,embeddings)

In [8]:
query= "I promise to each soldier that, upon the return from this expedition, he will have the means to buy six arpents5 of land."
result=db.similarity_search(query)
result[0].page_content

'Egypt, he was still seen as admirable and was crowned Emperor only a few short years after.     These 8 separate documents are all from the beginning of this legendary campaign, from an inspiration al \nspeech to his soldiers before leaving France, to the officia l \nstatement issued to the Egyptian people after his first attacks against the Mamluk forces, the Battle of the Pyramids. The tone in each text never fails to be optimisti c, \ncompassionate, or encouraging, even in his proclamations warning Egyptian officials of his impending actions. This \nseries of statements allows us to see the type of commanding officer that Napoleon Bonaparte truly was, where despite the fact that his personal greed for power was his driving force, he was capable of encouraging his  \ntroops and even the people he conquered to all be passionate for a common cause and to trust wholeheartedly  \nin their leader.'

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [21]:
prompt=ChatPromptTemplate.from_template(
        ("""You are an expert on Napolean Bonaparte and know all of his speeachs history and knowledge. So please give answer based on that.
        <context>
         {context}
         </context>
         Question:{input} 
         """),
        
)

### Chain 

In [22]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [23]:
retriver=db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000015AFFD3B1F0>)

In [24]:
from langchain.chains import create_retrieval_chain

retreval_chain = create_retrieval_chain(retriver,document_chain)

In [28]:
response = retreval_chain.invoke({"input":"Speech by Napolean to French Army"})

In [29]:
response['answer']

"My dear soldiers, as we stand on the eve of this great campaign, I prophesy to you that we shall emerge victorious, that our names will be etched in the annals of history alongside the greatest conquerors. We are not merely an army, but a force for change, a bulwark against tyranny and oppression.\n\nWe have come too far, faced too many challenges, and sacrificed too much to turn back now. Our path is clear: we shall march forth, our bayonets fixed, our hearts ablaze with the passion of liberty and equality. We shall not be deterred by the petty squabbles of politics or the whims of fate.\n\nWe are the vanguard of a new era, one in which the people shall be free to determine their own destiny. And we, my brave soldiers, shall be the instruments of that freedom. We shall not be swayed by the doubts and fears of others; we shall forge our own path, unafraid to take risks or defy convention.\n\nSo let us march forth, my friends, with confidence in ourselves, in our ideals, and in each ot